### Loading libs

In [1]:
### Testing simple mesh substraction

import os
import sys
import math
import gendata as gen
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
import cloudComPy as cc 
import psutil
from scipy import stats

print("Libs loaded")

Libs loaded


### Functions

In [2]:
def giveSpherePath(x:float = 0, y:float = 0, z:float = 0 , r:float = 10, num:int = 100, dx:float = 0.1, h:float = 5):
    """give_sphere Create a sphere point cloud
    Parameters"""
    dataDir = os.path.join(os.environ["USERPROFILE"], "CloudComPy/data")
    print("dataDir: ", dataDir)
    if not os.path.exists(dataDir):
        os.makedirs(dataDir)
    # points = []
    # for i in range(0, num):
    #     factor = stats.norm.pdf((np.arange(-5, 5, 0.001)), 0, 1)
    #     # factor = normedgauss()        # A value between 0 and 1 following a gaussian
    #     ir = r * factor
    #     itheta = np.arccos(np.random.uniform(-1, 1))
    #     iphi = np.random.uniform(0, 2 * np.pi)
    #     ix = x + ir * np.sin(itheta) * np.cos(iphi)
    #     iy = y + ir * np.sin(itheta) * np.sin(iphi)
    #     iz = z + ir * np.cos(itheta)
    #     points.append((ix, iy, iz))

    dataName = "dataSample_%s.xyz" % h
    if dx != 0:
        dataName = "dataSample_%s_%s.xyz" % (h,dx)
    sample = os.path.join(dataDir, dataName)
    vx = np.array([])
    vy = np.array([])
    if not os.path.exists(sample):
        print("Generating data: start X and Y!")
        for i in range(0, num):
            factor = stats.norm.pdf((np.arange(-5, 5, 0.001)), 0, 1)
            ir = r * factor
            itheta = np.arccos(np.random.uniform(-1, 1))
            iphi = np.random.uniform(0, 2 * np.pi)
            vx = np.append(arr = vx, values =  x + ir * np.sin(itheta) * np.cos(iphi))
            vy = np.append(arr = vy, values = y + ir * np.sin(itheta) * np.sin(iphi))
        print("Generating data: X and Y done!")

        with open(sample, 'w') as f:
            for xCoord in vx:
                for yCoord in vy:
                    zCoord = math.sin(h * math.sqrt(xCoord**2 + yCoord**2)) / math.sqrt(xCoord**2 + yCoord**2)
                    f.write("%f %f %f\n" % (xCoord, yCoord, zCoord))
                print("Generating data: Z done!")
    return sample

### Initialising cloud

In [3]:
# os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

gen.createSymbolicLinks() # required for tests on build, before cc.initCC.init
cc.initCC()

cloud = cc.loadPointCloud(gen.getSampleCloud2(step=0.1,dx=0.5))
# cloud = cc.loadPointCloud(giveSpherePath(num=5))

dataDir:  C:\Users\szinp\CloudComPy/data


### Creating primitives

In [4]:
# ccGLMatrix() is a transformation matrix?
tr1 = cc.ccGLMatrix()
tr1.initFromParameters(0.1, 0.2, 0.3, (8.0, 0.0, 0.0))
box = cc.ccBox((1., 2., 3.), tr1, "aBox")
if box.getName() != 'aBox':
    raise RuntimeError
if box.size() != 12:
    raise RuntimeError

tr5 = cc.ccGLMatrix()
tr5.initFromParameters(0.0, (0., 0., 0.), (-7.0, 5.0, 1.0))
sphere = cc.ccSphere(1.5, tr5, "aSphere", 72)
if sphere.getName() != 'aSphere':
    raise RuntimeError
if sphere.size() != 10224:
    raise RuntimeError

### Computing approximate Cloud distance to mesh
[]: # Language: python
[]: # Path: testingFunctions.ipynb

In [5]:
stats = cc.DistanceComputationTools.computeApproxCloud2MeshDistance(cloud, sphere)
print(stats) # min, max, mean, variance, error max
nbCpu = psutil.cpu_count()
bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(cloud,box)
params = cc.Cloud2MeshDistancesComputationParams()
params.maxThreadCount = nbCpu
params.octreeLevel = bestOctreeLevel
cc.DistanceComputationTools.computeCloud2MeshDistances(cloud, box, params)
bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(box.getAssociatedCloud(), sphere)
params.octreeLevel = bestOctreeLevel
cc.DistanceComputationTools.computeCloud2MeshDistances(sphere.getAssociatedCloud(), sphere, params)

[5.562787055969238, 9.822517395019531, 7.639709949493408, 0.4474430978298187, 0.030983304604887962]


1

In [6]:
cc.SaveEntities([cloud, box, sphere], os.path.join("", "entities2.bin"))

_cloudComPy.CC_FILE_ERROR.CC_FERR_NO_ERROR